


*   ***Rajat Kumar : 2048018***

*   ***Project on NLP (Part2)***
*   ***Youtube Comment Classification***








In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_excel("nlp_data.xlsx")

In [3]:
import nltk 
from nltk.corpus import stopwords
from bs4 import BeautifulSoup 

import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV


In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# ***Text Cleaning/Tokenization***

**Note:** Beautiful Soup is a Python library for pulling data out of HTML and XML files. It works with your favorite parser to provide idiomatic ways of navigating, searching, and modifying the parse tree. It commonly saves programmers hours or days of work.

In [5]:
# Function for cleaning of text data, converting to lower case and finally splitting
def cleanText(rawText):
  temp = BeautifulSoup(rawText, "html.parser")
  letters_only = re.sub("[!\"#\$%&\\\\'\(\)\*\+\,\-\./:;<\=>?@\[\]\^_`\{|\}~0-9]", "", temp.get_text())
  lower_case = letters_only.lower()
  words = lower_case.split()
  words = [w for w in words if w not in stopwords.words("english")]
  return(" ".join(words))

In [6]:
cleanX = []
for i in df.text:
  cleanX.append(cleanText(i))

/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "https://www.manoramaonline.com/homestyle/spot-light/2019/10/31/lekshmi-nair-house-in-kochi-celebrity-home.html" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [7]:
len(cleanX)

4291

In [8]:
print(cleanX)

['തേങ്ങ തേങ്ങാപ്പാൽ ഈസ്റ്റ് ഇവയൊന്നും ചേർത്തത് കാണിച്ചില്ലല്ലോ', 'thank kaima rice doubt clear aayi eppol', 'വീണ ചേച്ചി ബ്രെഡ് ഒമ്ബ്ലെയ്ട്', 'happy journey', 'u come back mam', 'love story veena thanks sharing made feel like became family member keep watching whenever comes actually watched video ago thinking christian testimony always watch changed lives many real life watch videos pops almost everyday love see family often simple way sharing love jan brother simple young man role model great example many respectappreciation love jans parents sweet hear praying god take care family good see blessednever quit stayed strong difficult time life really love listen beautiful vision avi born seeing great hope livegod plan great purpose family athu added blessing take care love stay blessed', 'waiting recipe u thanks lot giving recipe', 'thank much chechi perfect', 'helpful chechii ennepolulla begginers nu', 'swabavama ithu nalalu kanan undankil oru virikala pinne oru virappikal pinna onnu v

In [9]:
# Checking how nltk works with a mixed comment
sample_text = "Njaan sister nte video, just saw it by coincidence! Muzuhvan story irunnu kettu. Wow, big salute to you for what you went through and your husband Jan for accepting you into his life! Valiya namaskaram to his parents.Greetings from Belgium"

nltk.download("words")
words = set(nltk.corpus.words.words())
ans = " ".join(w for w in nltk.wordpunct_tokenize(sample_text) if w.lower() in words or not w.isalpha())

print(ans)

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
sister video , just saw it by coincidence ! story . Wow , big salute to you for what you went through and your husband for you into his life ! to his . from




*   For the mixed comment nltk is doing the job somewhat 
averagely, needs a lot of improvement on multilingual comments as most of the malyalam words are filtered out without translation




# ***Feature Extraction/ Vectorization***

**Note:** CountVectorizer is used to convert a collection of text documents to a vector of term/token counts. It also enables the ​pre-processing of text data prior to generating the vector representation. This functionality makes it a highly flexible feature representation module for text.

In [10]:
vectorizer = CountVectorizer(analyzer = "word",
                            preprocessor = None,
                            stop_words = "english",
                            max_features = 5000)

In [11]:
train_data_features = vectorizer.fit_transform(cleanX)
train_data_features = train_data_features.toarray()
print(train_data_features)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [12]:
x_col = train_data_features
y_col = df.label

In [13]:
print(y_col.shape)
print(x_col.shape)

(4291,)
(4291, 5000)


# ***Model Building***

In [14]:
x_train, x_test ,y_train, y_test = train_test_split(x_col, y_col, test_size = 0.3, random_state = 42)

***M-Naive Bayes Classifier***

In [15]:
naive = MultinomialNB()
classifier = naive.fit(x_train, y_train)
predict_nb = classifier.predict(x_test)

In [16]:
import sklearn

In [17]:
nb_acc = sklearn.metrics.accuracy_score(y_test, predict_nb)
nb_mcc = sklearn.metrics.matthews_corrcoef(y_test, predict_nb)

In [18]:
print(predict_nb)
print(nb_acc)
print(nb_mcc)

[6 2 6 ... 7 6 6]
0.5861801242236024
0.3633921370829238


**SVM Classifier**

In [19]:
from sklearn import svm
#This strategy consists in fitting one classifier per class pair. 
#At prediction time, the class which received the most votes is selected.
#This method may be advantageous for algorithms such as kernel algorithms which don’t scale well with n_samples
clf = svm.SVC(decision_function_shape = "ovo")
clf.fit(x_train, y_train)
predict_svm = clf.predict(x_test)

In [20]:
svm_acc = sklearn.metrics.accuracy_score(y_test, predict_svm)
svm_mcc = sklearn.metrics.matthews_corrcoef(y_test, predict_svm)


In [21]:
print(predict_svm)
print(svm_acc)
print(svm_mcc)

[6 2 6 ... 6 6 4]
0.6110248447204969
0.38474844621730253


***KNN Classifier***

In [22]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors = 5)
neigh.fit(x_train, y_train)
predict_knn = neigh.predict(x_test)

In [23]:
knn_acc = sklearn.metrics.accuracy_score(y_test, predict_knn)
knn_mcc = sklearn.metrics.matthews_corrcoef(y_test, predict_knn)

In [24]:
print(predict_knn)
print(knn_acc)
print(knn_mcc)

[6 2 2 ... 4 4 4]
0.5427018633540373
0.29594840385599835


In [26]:
# Parameter tuning to check effect of n_neighbors on accuracy
parameter_space = {'n_neighbors':[1,3,7,9]}
          
neigh = GridSearchCV(neigh, parameter_space, n_jobs=-1, cv=5)
grid_result=neigh.fit(x_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.544785 using {'n_neighbors': 9}
0.525809 (0.024574) with: {'n_neighbors': 1}
0.519154 (0.021925) with: {'n_neighbors': 3}
0.541791 (0.013474) with: {'n_neighbors': 7}
0.544785 (0.009709) with: {'n_neighbors': 9}


No specific pattern is visible as the accuracy is fluctuating

***Random Forest Classifier***

In [27]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 1000)
rf.fit(x_train, y_train)
predict_rf = rf.predict(x_test)

rf_acc = sklearn.metrics.accuracy_score(y_test, predict_rf)
rf_mcc = sklearn.metrics.matthews_corrcoef(y_test, predict_rf)

print(predict_rf)
print(rf_acc)
print(rf_mcc)

[6 2 6 ... 4 6 4]
0.6203416149068323
0.43041253228529075


In [ ]:
#Parameter tuning to check effect of n_estimators on accuracy
parameter_space =  {'n_estimators':[1500,2000,2500]}
rf1 = GridSearchCV(rf, parameter_space, n_jobs=-1, cv=5)
grid_result=rf1.fit(x_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [27]:
result = {'Model': ['M-Naive Bayes Classifier','SVM Classifier','KNN Classifier','Random Forest Classifier '],
        'MCC': [0.363,0.384,0.295,0.431],
        'ACC(%)':[58.61,61.10,54.27,62.18]
        }

df1 = pd.DataFrame(result, columns = ['Model', 'MCC', 'ACC(%)'])

df1.head()

,Model,MCC,ACC(%)
0,M-Naive Bayes Classifier,0.363,58.61
1,SVM Classifier,0.384,61.10
2,KNN Classifier,0.295,54.27
3,Random Forest Classifier,0.431,62.18


***From the above Results we can clearly see thar Random Forest is giving the best Accuracy and best Mathews Corr Coeff, although SVM is also performing almost similarly.***